In [1]:
import logging
from time import sleep

from codenames.game import QuitGame

from playground.boards import *  # noqa
from utils import configure_logging

configure_logging()

log = logging.getLogger(__name__)

In [23]:
from solvers.models import ModelIdentifier, HEBREW_SUFFIX_ADAPTER, load_model_async, DEFAULT_MODEL_ADAPTER
from codenames.game import GameRunner
from solvers.naive import NaiveHinter, NaiveGuesser

model_id = ModelIdentifier(language="english", model_name="wiki-50", is_stemmed=False)
# model_id = ModelIdentifier("english", "google-300", False)
# model_id = ModelIdentifier("hebrew", "twitter", False)
# model_id = ModelIdentifier("hebrew", "ft-200", False)
# model_id = ModelIdentifier(language="hebrew", model_name="skv-ft-150", is_stemmed=True)
# model_id = ModelIdentifier(language="hebrew", model_name="skv-cbow-150", is_stemmed=True)
# model_id = ModelIdentifier("hebrew", "skv-cbow-150", True)

# os.environ[MODEL_NAME_ENV_KEY] = model_id.model_name
# os.environ[IS_STEMMED_ENV_KEY] = "1" if model_id.is_stemmed else ""
adapter = HEBREW_SUFFIX_ADAPTER if model_id.language == "hebrew" and model_id.is_stemmed else DEFAULT_MODEL_ADAPTER
load_model_async(model_id)

blue_hinter = NaiveHinter("Leonardo", model_identifier=model_id, model_adapter=adapter)
blue_guesser = NaiveGuesser("Bard", model_identifier=model_id, model_adapter=adapter)
red_hinter = NaiveHinter("Adam", model_identifier=model_id, model_adapter=adapter)
red_guesser = NaiveGuesser("Eve", model_identifier=model_id, model_adapter=adapter)

game_runner = GameRunner(blue_hinter, red_hinter, blue_guesser, red_guesser)

In [ ]:
r = game_runner.run_game()

In [22]:
# game_runner.run_game(language="english", board=english_board)
result = game_runner.run_game(language="hebrew", board=HEBREW_BOARD_1)

Winner(team_color=<TeamColor.RED: 'Red'>, reason=<WinningReason.TARGET_SCORE_REACHED: 'Target score reached'>)

In [ ]:
from codenames.online import NamecodingGameRunner, NamecodingLanguage

online_runner = None
try:
    online_runner = NamecodingGameRunner(blue_hinter, red_hinter, blue_guesser, red_guesser)
    online_runner.auto_start(language=NamecodingLanguage.ENGLISH, clock=False)
    sleep(1)
except QuitGame:
    pass
except Exception as e:  # noqa
    log.exception("Error occurred")
finally:
    if online_runner is not None:
        log.info(f"Winner: {online_runner.winner}")
        online_runner.close()
log.info("Done")

In [6]:
from solvers.utils.loader import load_model
from codenames.boards.hebrew import *  # noqa

# for word in HEBREW_WORDS:
#     for other in HEBREW_WORDS:
#         if word != other and editdistance.eval(word, other) <= 2 and abs(len(word) - len(other)) >= 2:
#             print(f"{word} {other}")

print(f"Got {len(HEBREW_WORDS)} words")
model = load_model(model_id)
errors = []
for word in HEBREW_WORDS:
    # log.info("Checking %s", word)
    try:
        x = model[adapter.to_model_format(word)]
    except Exception as e:
        errors.append(word)
print("Done with %d errors" % len(errors))

Got 424 words
Loading model language='hebrew' model_name='skv-cbow-30' is_stemmed=True
Done with 0 errors
